In [ ]:
# default_exp pydecorator

# Python decorators

> Some examples on how to use decorator in python.

Tutorial: Geir Arne Hjelle - Introduction to Decorators: Power Up Your Python Code. Link: https://www.youtube.com/watch?v=T8CQwGIsrx4

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import functools
import logging
import random
import time
from collections import defaultdict

### Functions are first class objects

In [ ]:
#export
# Function as a parameter
def hello(name, logger):
    logger(f"Hello {name}")
    
hello("world", logger=print)

Hello world


In [ ]:
hello("First log",logger=logging.warning)

In [ ]:
hello("Second log",logger=logging.info)

In [ ]:
with open("hello.txt", mode="w") as file:
    hello("Files", logger=file.write)
!cat hello.txt

In [ ]:
def reversed_print(text):
    print(text[::-1].capitalize())

reversed_print("Hi there")

Ereht ih


In [ ]:
hello("Danh", logger=reversed_print)

Hnad olleh


In [ ]:
hello("world", logger=print)

Hello world


### Inner functions

In [ ]:

def outer():
    print("Hi from the outer")
    y = 2020
    def inner():
        print("Hello from the inner")
        print(f"This year is {y}")
    inner()
    return inner

In [ ]:
outer()

Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [ ]:
inside = outer()
inside

Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [ ]:
inside()

Hello from the inner
This year is 2020


#### Manipulate functions

In [ ]:
def hi(func):
    print(f"Hello {func.__name__}")
hi(outer)

Hello outer


In [ ]:
inside

<function __main__.outer.<locals>.inner()>

In [ ]:
hi(inside)

Hello inner


In [ ]:
def hi(func):
    print(f"Hello {func.__name__}")
    return func

In [ ]:
hi(outer)()

Hello outer
Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [ ]:
new_outer = hi(outer)

Hello outer


In [ ]:
new_outer is outer

True

In [ ]:
def wrapper(func):
    def _wrapper():
        print(f"Before {func.__name__}")
        func()
        print(f"After {func.__name__}")
    return _wrapper

wrapper(outer)

<function __main__.wrapper.<locals>._wrapper()>

In [ ]:
outer()

Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [ ]:
new_outer = wrapper(outer)


In [ ]:
new_outer()

Before outer
Hi from the outer
Hello from the inner
This year is 2020
After outer


In [ ]:
outer = wrapper(outer)

In [ ]:
outer

<function __main__.wrapper.<locals>._wrapper()>

In [ ]:
outer()

Before outer
Hi from the outer
Hello from the inner
This year is 2020
After outer


#### Sugar syntax

In [ ]:
# Same as: outer2 = wrapper(outer2)
@wrapper
def outer2():
    print("Hi from the outer")
    y = 2020
    def inner():
        print("Hello from the inner")
        print(f"This year is {y}")
    inner()
    return inner

In [ ]:
outer2()

Before outer2
Hi from the outer
Hello from the inner
This year is 2020
After outer2


In [ ]:
@wrapper
def dice_roll():
    return random.randint(1,6)

In [ ]:
dice_roll()

Before dice_roll
After dice_roll


In [ ]:
dice_roll.__name__

'_wrapper'

In [ ]:
def wrapper(func):
    def _wrapper(*args, **kwargs):
        print(f"Before {func.__name__}")
        func(*args, **kwargs)
        print(f"After {func.__name__}")
    return _wrapper

@wrapper
def hello(name):
    print(f"Hello {name}")

hello("world")

Before hello
Hello world
After hello


In [ ]:
def wrapper(func):
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        print(f"Before {func.__name__}")
        value = func(*args, **kwargs)
        print(f"After {func.__name__}")
        return value
#    _wrapper.__name__ = func.__name__
    return _wrapper

@wrapper
def dice_roll():
    """ Roll a 6-sided dice"""
    return random.randint(1,6)

dice_roll()

Before dice_roll
After dice_roll


4

In [ ]:
dice_roll.__name__

'dice_roll'

### Task 1: Time counter

In [ ]:
#export
def wrapper(func):    
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        # Before func
        value = func(*args, **kwargs)
        # After func
        return value
    return _wrapper

In [ ]:
#export
def timer(func):
    @functools.wraps(func)
    def _timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        print(f"Elapsed time: {toc-tic:.2f} seconds")
        return value
    return _timer


@timer
def waste_time(number):
    total = 0
    for num in range(number):
        total += sum(n for n in range(num))
    return total

In [ ]:
# Test
waste_time(300)

Elapsed time: 0.02 seconds


4455100

In [ ]:
waste_time(3000)

Elapsed time: 0.44 seconds


4495501000

### Task 2: trace

In [ ]:
#export

def get_params(*args, **kwargs):
    ars = [repr(a) for a in args]
    kws = [f"{k}={repr(v)}" for k,v in kwargs.items()]
    return ', '.join(ars + kws)

def trace(func):
    """Show the trace of function calls"""
    name = func.__name__
    @functools.wraps(func)
    def _trace(*args, **kwargs):
        print(f"Calling {name}({get_params(*args,**kwargs)})")
        value = func(*args, **kwargs)
        print(f"{name} returned {value}")
        return value
    return _trace

GREETINGS = ["ABC", "EHLLO", "NO!!!"]

@trace
def greet(name, greeting="Hello"):
    return f"{greeting} {name}"


@trace
@timer
def random_greet(name="Emily"):
    greeting = random.choice(GREETINGS)
    return greet(name, greeting=greeting)

@trace
def greet_many(number):
    return [random_greet() for _ in range(number)]


In [ ]:
greet("world")

Calling greet('world')
greet returned Hello world


'Hello world'

In [ ]:
greet(name="world", greeting="def")

Calling greet(name='world', greeting='def')
greet returned def world


'def world'

In [ ]:
random_greet()

Calling random_greet()
Calling greet('Emily', greeting='ABC')
greet returned ABC Emily
Elapsed time: 0.00 seconds
random_greet returned ABC Emily


'ABC Emily'

In [ ]:
greet_many(3)

Calling greet_many(3)
Calling random_greet()
Calling greet('Emily', greeting='ABC')
greet returned ABC Emily
Elapsed time: 0.00 seconds
random_greet returned ABC Emily
Calling random_greet()
Calling greet('Emily', greeting='ABC')
greet returned ABC Emily
Elapsed time: 0.00 seconds
random_greet returned ABC Emily
Calling random_greet()
Calling greet('Emily', greeting='ABC')
greet returned ABC Emily
Elapsed time: 0.00 seconds
random_greet returned ABC Emily
greet_many returned ['ABC Emily', 'ABC Emily', 'ABC Emily']


['ABC Emily', 'ABC Emily', 'ABC Emily']

In [ ]:
random_greet.__name__

'random_greet'

### TASK 3: Register

In [ ]:
#export
REGISTERED = {}

def register(func):
    name = func.__name__
    if name not in REGISTERED: REGISTERED[name] = func
    return func

@register
def true_or_false(text):
    tf_values = {
        True: {"true", "on", "yes", "1"},
        False: {"false", "off", "no", "0"}
    }
    for tf, values in tf_values.items():
        if text.lower() in values:
            return tf

@register
def reversed(text):
    return text[::-1].capitalize()

@register
def robber_language(text):
    consonants = "bcdfghlmnpqrstvwxyz"
    return "".join(
        f"{c}o{c.lower()}" if c.lower() in consonants else c
        for c in text
    )

# text = input("Please input a text:")

# while True:
#     print(f"Parsers: {', '.join(REGISTERED)}")
#     parser = input("Choose a parser: ")
#     if parser in REGISTERED: break


# parser_func = REGISTERED[parser]
# print(parser_func(text))
# parser, text, REGISTERED[parser]

In [ ]:
REGISTERED['robber_language']("decorator")

'dodecocororatotoror'

In [ ]:
words = ['map', 'mid', 'acb', 'gqre', 'hello', 'anh', 'minh']
ls = {}

for w in words:
    if w[0] not in ls.keys(): ls[w[0]] = [w]
    else: ls[w[0]].append(w)
ls

{'m': ['map', 'mid', 'minh'],
 'a': ['acb', 'anh'],
 'g': ['gqre'],
 'h': ['hello']}

In [ ]:
ls2 = {}
for w in words:
    ls2.setdefault(w[0],[]).append(w)

In [ ]:
def test_eq(x,y):
        assert x == y

In [ ]:
test_eq(ls2, ls)

In [ ]:
def get_second(w): return w[1] if len(w) > 1 else w


In [ ]:
ls3 = {}
for w in words:
    ls3.setdefault(get_second(w),[]).append(w)

In [ ]:
a = '1'
hash(a)

-5838634506333221685

In [ ]:
hash(tuple(set(dir(dict))))

-6058033488329309327

In [ ]:

def get_dict1(words):
    ls2 = {}
    for w in words:
        ls2.setdefault(w[0],[]).append(w)
    return ls2

def get_dict2(words):
    ls4 = defaultdict(list)
    for w in words:
        ls4[w[0]].append(w)
    return ls4

test_eq(get_dict1(words),get_dict2(words))

In [ ]:
%timeit get_dict1(words)

2.95 µs ± 587 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
%timeit get_dict2(words)

2.85 µs ± 565 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
count = defaultdict(lambda : 4)
type(count)

In [ ]:
count[2] = 23

In [ ]:
count[3], count

defaultdict(<function __main__.<lambda>()>, {2: 23})

In [ ]:
#hide
# Convert to *.py
from nbdev.export import notebook2script
notebook2script()

Converted 01_pydecorator.ipynb.
Converted 02_pyscraper.ipynb.
Converted 03_pyencoding.ipynb.
Converted 04_housing_data_collection.ipynb.
Converted 05_pytwitter.ipynb.
Converted 06_skorch.ipynb.
Converted 07_math_optimisation.ipynb.
Converted 08_ds_scratch.ipynb.
Converted 09_openstreet_map.ipynb.
Converted 20_dash_01.ipynb.
Converted index.ipynb.
